In [1]:
from tensorflow.contrib.layers import variance_scaling_initializer
import tensorflow as tf

be_init = variance_scaling_initializer()

In [2]:
def conv(bottom,
        num_filter, ksize=3, stride=1, padding="SAME",
        scope=None):
    bottom_shape = bottom.get_shape().as_list()[3]
    
    with tf.variable_scope(scope or "conv"):
        W = tf.get_variable("W",
                           [ksize, ksize, bottom_shape, num_filter],
                           initializer=he_init)
        b = tf.get_variable("b", [num_filter],
                           initializer=tf.constant_initializer(0))
        x = tf.nn.conv2d(bottom, W,
                        strides=[1, stride, stride, 1],
                        padding=padding)
        x = tf.nn.relu(tf.nn.bias_add(x, b))
    return x

def maxpool(bottom, ksize=2, stride=2, padding="SAME", scope=None):
    with tf.variable_scope(scope or "maxpool"):
        pool = tf.nn.max_pool(bottom, ksize[1, ksize, ksize, 1],
                             strides=[1, stride, stride, 1],
                             padding = padding)
    return pool

def fc(bottom, num_dims, scope=None):
    bottom_shape = bottom.get_shape().as_list()
    if len(bottom_shape) > 2:
        bottom = tf.reshape(bottom,
                           [-1, reduce(lambda x, y: x*y, bottom_shape[1:])])
        bottom_shape = bottom.get_shape().as_list()
    with tf.variable_scope(scope or "fc"):
        W = tf.get_variable("W", [bottom_shape[1], num_dims],
                           initializer = he_init)
        b = tf.get_variable("b", [num_dims],
                           initializer=tf.constant_initializer(0))
        out = tf.nn.bias_add(tf.matmul(bottom, W), b)
    return out

def fc_relu(bottom, num_dims, scope=None):
    with tf.variable_scope(scope or "fc"):
        out = fc(bottom, num_dims, scope="fc")
        relu = tf.nn.relu(out)
    
    return relu



In [3]:
keep_prob = tf.placeholder(tf.float32, None)

In [4]:
def conv_net(x, keep_prob):
    x = tf.reshape(x, shape=[-1, 28, 28, 1])
    
    conv1 = conv(x, 32, 5, scope="conv_1")
    conv1 = maxpool(conv1, scope="maxpool_1")
    conv2 = conv(conv1, 64, 5, scope="conv_2")
    conv2 = maxpool(conv2, scope="maxpool_2")
    
    fc1 = fc_relu(conv2, 1024, scope="fc_1")
    fc1 = tf.nn.dropout(fc1, keep_prob)
    
    out = fc(fc1, 10, scope="out")
    return out

## 12. Session config

In [12]:
config = tf.ConfigProto(gpu_options = tf.GPUOptions(allow_growth=False))
sess = tf.Session(config = config)

## 13. Layers

In [13]:
input = ...
with tf.name_scope('conv1_1') as scope:
    kernel = tf.Variable(tf.truncated_normal([3,3,64,128],
                                            dtype=tf.float32, stddev=1e-1),
                        name='weights')
    conv = tf.nn.conv2d(input, kernel, [1,1,1,1], padding="SAME")
    biases = tf.Variable(tf.constant(0.0, shape=[128],
                                    dtype=tf.float32),
                        trainable=True, name='biases')
    bias = tf.nn.bias_add(conv, biases)
    conv1 = tf.nn.relu(bias, name=scope)

TypeError: Expected binary or unicode string, got Ellipsis